In [ ]:
df_subset = df[['TIMESTAMP', 'M_AIR_TEMPERATURE','M_TIME_OFFSET'] + df.columns.tolist()[31:39]]
'rows with na:', df_subset.shape[0] - df_subset.dropna().shape[0]

In [ ]:
from sklearn import preprocessing

x = df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_norm = pd.DataFrame(x_scaled, columns=df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).columns.tolist())
df_norm['TIMESTAMP'] = pd.to_datetime(df_subset['TIMESTAMP'], unit='s', dayfirst=True).reset_index(drop=True)
df_norm['M_WEATHER'] = df_subset['M_WEATHER'].reset_index(drop=True)
df_norm = df_norm.set_index('TIMESTAMP')
df_norm

In [ ]:
df_norm['M_WEATHER'] = df_norm['M_WEATHER'].apply(lambda x: str(x))
df_norm

In [ ]:
df_norm.dropna(inplace=True)

In [ ]:
df = df_norm
Y = df['M_RAIN_PERCENTAGE']
X = df.drop(columns=['M_RAIN_PERCENTAGE'],inplace=False)

In [ ]:
X, X_test, y, y_test = train_test_split(X,Y,test_size = 0.2,train_size = 0.8,random_state = 42)
X_train, X_cv, y_train, y_cv = train_test_split(X,y,test_size = 0.25,train_size = 0.75,random_state = 42)

In [ ]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
model.fit(X_train,y_train)

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
n_scores = cross_val_score(model, X_cv, y_cv, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
predictions = model.predict(X_test)
MSE = mean_squared_error(y_test,predictions)
print(MSE)

In [ ]:
row = X.iloc[1].to_numpy().reshape(1,-1)
yhat = model.predict(row)
print('predicted rain probability is',yhat)

In [ ]:
filename = "finalized_randomforest_regressor_model.sav"
pickle.dump(model, (open(filename, "wb")))